# 🎬 Hybrid Mood Classifier for Netflix Dataset

In [1]:
import pandas as pd
from transformers import pipeline

In [5]:
# Load Netflix dataset
df = pd.read_csv("C:/Users/Vanathi/mood-meets-media/data/external/netflix_titles.csv", encoding="ISO-8859-1")
df = df[["title", "type", "listed_in", "description"]].dropna()


In [6]:
# Load Hugging Face classifier
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)

In [9]:
# Define keyword-based and hybrid mood detection functions
def keyword_mood(desc):
    desc = desc.lower()
    if "thriller" in desc or "dark" in desc or "survival" in desc:
        return "Tense"
    elif "romantic" in desc or "love" in desc:
        return "Happy"
    elif "tragedy" in desc or "death" in desc or "loss" in desc:
        return "Sad"
    elif "funny" in desc or "comedy" in desc or "laugh" in desc:
        return "Happy"
    elif "mystery" in desc or "investigation" in desc:
        return "Anxious"
    elif "exciting" in desc or "adventure" in desc:
        return "Excited"
    return "Neutral"

def hybrid_mood(row):
    desc_mood = keyword_mood(row['description'])
    if desc_mood == "Neutral":
        result = classifier(row['description'][:512])
        return result[0][0]['label']
    return desc_mood

In [10]:
# Apply hybrid mood classifier
df["mood_tag"] = df.apply(hybrid_mood, axis=1)

In [17]:
# Align mood_tag with mood_logs schema
mood_standard_map = {
    "joy": "Happy",
    "love": "Happy",
    "surprise": "Excited",
    "anger": "Stressed",
    "fear": "Anxious",
    "sadness": "Sad",
    
    # Hybrid classifier additions
    "Happy": "Happy",
    "Sad": "Sad",
    "Excited": "Excited",
    "Anxious": "Anxious",
    "Tense": "Stressed",
    "Stressed": "Stressed",
    "Neutral": "Neutral",
    "neutral": "Neutral",
    "disgust": "Stressed"
}


In [18]:
# Apply to your Netflix dataframe
df["mood_tag_standard"] = df["mood_tag"].map(mood_standard_map)


In [19]:
print(df["mood_tag"].unique())
print(df["mood_tag_standard"].unique())

['Sad' 'disgust' 'anger' 'neutral' 'fear' 'Anxious' 'Excited' 'Happy'
 'sadness' 'joy' 'Tense' 'surprise']
['Sad' 'Stressed' 'Neutral' 'Anxious' 'Excited' 'Happy']


In [20]:
# Format and save
df = df.rename(columns={"listed_in": "genre", "description": "tags"})
df.to_csv("C:/Users/Vanathi/mood-meets-media/data/processed/netflix_enriched_metadata_hybrid.csv", index=False)
print("✅ Hybrid mood-tagged metadata saved to: data/processed/netflix_enriched_metadata_hybrid.csv")

✅ Hybrid mood-tagged metadata saved to: data/processed/netflix_enriched_metadata_hybrid.csv
